In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig

d:\gsf\green-software-foundation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset

In [4]:
from trl import SFTTrainer

In [5]:
from jinja2 import Template

In [6]:
import yaml

In [7]:
MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"

In [8]:
NEW_MODEL_NAME = "TinyButMighty"

In [9]:
DATASET_NAME = "macadeliccc/opus_samantha"

In [10]:
SPLIT = "train"

In [11]:
MAX_SEQ_LENGTH = 2048

In [12]:
num_train_epochs = 1

In [13]:
license = "apache-2.0"

In [14]:
username = "NamanAhuja"

In [15]:
learning_rate = 1.41e-5

In [16]:
per_device_train_batch_size = 4

In [17]:
gradient_accumulation_steps = 1

In [18]:
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, trust_remote_code=True)

d:\gsf\green-software-foundation\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

In [ ]:
dataset = load_dataset("macadeliccc/opus_samantha", split="train")

In [ ]:
EOS_TOKEN=tokenizer.eos_token_id

In [ ]:
def process_dataset(mydata):

    conversations = mydata["conversations"]

    texts = []

    mapper = {"system": "system\n", "human": "\nuser\n", "gpt": "\nassistant\n"}

    end_mapper = {"system": "", "human": "", "gpt": ""}

    for c in conversations:

        text = "".join(f"{mapper[(turn := x['from'])]} {x['value']}\n{end_mapper[turn]}" for x in c)

        texts.append(f"{text}{EOS_TOKEN}")

    return {"text": texts}

In [ ]:
dataset = dataset.map(process_dataset, batched=True)

In [ ]:
print(dataset['text'][2])

In [ ]:
args = TrainingArguments(

    per_device_train_batch_size=1,

    gradient_accumulation_steps=gradient_accumulation_steps,

    gradient_checkpointing=True,

    learning_rate=2e-5,

    lr_scheduler_type="cosine",

    max_steps=-1,

    num_train_epochs=num_train_epochs,

    save_strategy="no",

    logging_steps=1,

    output_dir=NEW_MODEL_NAME,

    optim="paged_adamw_32bit",

    bf16=True,

In [ ]:
)

In [ ]:
trainer = SFTTrainer(

    model=model,

    args=args,

    train_dataset=dataset,

    dataset_text_field="text",

    max_seq_length=MAX_SEQ_LENGTH,

    formatting_func=process_dataset

In [ ]:
)

In [ ]:
trainer.train()